In [1]:
# Install required packages (run this cell if not already installed)
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl datasets pypdf langchain-community langchain-huggingface ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import os
import pandas as pd


In [3]:
from tqdm.notebook import tqdm
from typing import Optional, List, Tuple
from datasets import Dataset
from langchain.document_loaders import DirectoryLoader, PyPDFLoader


In [ ]:
# down mergrng data all pdf fils and metadata from csv files to docs


In [4]:


def load_resumes(data_dir: str, csv_path: str = None) -> list:
    """Load resumes from directory and optional CSV metadata."""
    # Load CSV if provided (single line with error handling)
    df = pd.read_csv(csv_path) if csv_path and os.path.exists(csv_path) else None
    
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Directory not found: {data_dir}")
    
    documents = []
    for category in os.listdir(data_dir):
        category_path = os.path.join(data_dir, category)
        if not os.path.isdir(category_path):
            continue
            
        # Load PDFs (removed progress bar for speed)
        loader = DirectoryLoader(category_path, glob="*.pdf", loader_cls=PyPDFLoader)
        docs = loader.load()
        
        for doc in docs:
            # Simplified metadata extraction
            doc.metadata.update({
                "category": category,
                "file_name": os.path.basename(doc.metadata["source"]),
                "id": os.path.splitext(doc.metadata["source"].split('/')[-1])[0]
            })
            
            # Optional CSV merge (one-liner)
            if df is not None:
                if match := df[df["ID"] == doc.metadata["id"]].to_dict('records'):
                    doc.metadata.update(match[0])
        
        documents.extend(docs)
    print(documents[0])
    return documents

# Usage (2 lines)
DATA_DIR = "/kaggle/input/resume-dataset/data/data"
docs = load_resumes(DATA_DIR, "/kaggle/input/resume-dataset/Resume/Resume.csv")

page_content='PRE-PRESS GRAPHIC DESIGNER
Summary
Creative, hardworking designer seeking a full-time desktop job, educated as a graphic artist, past experience in business world as a desktop
publisher laying out designs for printed mail and advertisements, in local government designing new websites with graphics for different agencies
within the system, and later for the same government printing and reproduction center creating documents to be printed off a press or copiers.
Skills
Adobe InDesign, Photoshop, Illustrator, and Acrobat Professional
Strongly familiar with Microsoft Word, Excel, PowerPoint, and Publisher / also QuarkXPress
Basic knowledge of web development with Adobe Dreamweaver, HTML, WordPress
Able to perform graphic design and administrative functions
Able to work as a team player and independently
Experienced using phone, fax, email, copiers and printers
Provides excellent customer service (in-person, by phone, email, or interoffice mail)
Prioritizes and calmly handles 

In [7]:
print(docs[0].page_content)  # First document in the dataset

PRE-PRESS GRAPHIC DESIGNER
Summary
Creative, hardworking designer seeking a full-time desktop job, educated as a graphic artist, past experience in business world as a desktop
publisher laying out designs for printed mail and advertisements, in local government designing new websites with graphics for different agencies
within the system, and later for the same government printing and reproduction center creating documents to be printed off a press or copiers.
Skills
Adobe InDesign, Photoshop, Illustrator, and Acrobat Professional
Strongly familiar with Microsoft Word, Excel, PowerPoint, and Publisher / also QuarkXPress
Basic knowledge of web development with Adobe Dreamweaver, HTML, WordPress
Able to perform graphic design and administrative functions
Able to work as a team player and independently
Experienced using phone, fax, email, copiers and printers
Provides excellent customer service (in-person, by phone, email, or interoffice mail)
Prioritizes and calmly handles multiple proje

In [9]:
# to create a raw k_b to feed into text spillter
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc.page_content, metadata={"Category": doc.metadata["category"]}) for doc in docs
]

  0%|          | 0/4881 [00:00<?, ?it/s]

KeyError: 'Category'

In [12]:
# splitting raw d_b to feed into embeddoing model so we can create vector db
from langchain.text_splitter import RecursiveCharacterTextSplitter

# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = [] # this splitts the charectters into chuks and store
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

In [13]:
# choosng andloadng emeddng model
from sentence_transformers import SentenceTransformer

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}")

from transformers import AutoTokenizer# model toknze the splited data

tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small") # here chuks get tokenized for vectors storage
lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]

2025-05-20 13:36:20.754483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747748180.943395      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747748180.995532      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model's maximum sequence length: 512


  0%|          | 0/18499 [00:00<?, ?it/s]

In [14]:
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [15]:
from langchain.vectorstores import FAISS # stores vector databse
from langchain_community.embeddings import HuggingFaceEmbeddings # fuction chain to perform embeddngs
from langchain_community.vectorstores.utils import DistanceStrategy # for retriver to identify smlar docs

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)
# here we hve xomplete vector knoelgde base to perform query to retrive docs or perform llm to
#generte ansrs

/tmp/ipykernel_31/2775243909.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
2025-05-20 13:37:40.021693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747748260.043264     126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747748260.049905     126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has 

Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
KNOWLEDGE_VECTOR_DATABASE # high dimension matrx 

In [17]:
# its a doc retriver for vector db
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={"k": 30})

In [18]:
docs = retriever.invoke("""
  "Machine Learning Engineer" OR "Data Scientist" AND (
    ("Python" AND ("Scikit-learn" OR "TensorFlow" OR "PyTorch"))  
    ("SQL" AND ("Spark" OR "Hadoop" OR "ETL"))  
    ("AWS" OR "GCP" OR "Azure" OR "MLOps")  
    ("Tableau" OR "Power BI" OR "data visualization")  
    ("statistical analysis" OR "A/B testing")  
    ("NLP" OR "LLM" OR "GenAI" OR "recommendation systems")  
  )  
  NOT ("intern" OR "student")  
  Years: "3+ years"  
  Location: "Remote" OR "Mumbai" OR "Bangalore" OR "Pune"  
""")

2025-05-20 13:38:43.334457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747748323.356176     146 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747748323.362945     146 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
import pprint

In [20]:
pprint.pp(docs) # retrved 30 docs to lter perform rerank

[Document(id='fcff8e39-33dc-4ec9-8ef6-31c1b9c77493', metadata={'Category': 'ENGINEERING', 'start_index': 917}, page_content='implement statistical / predictive models and cutting edge algorithms utilizing diverse sources of data to predict Diversified experience with\nEngineering, Manufacturing, Retailing, Higher Education and IT /Data related companies Fascinated by learning cutting edge technologies, such as;\nData Mining and Machine Learning Handled a team of 4 during my 3 years of Tech experience and managed more than 200 students and\neffectively evaluated the performance of each student and utilize assessment methods to judge overall progress during my teaching experience.\nQualifications\nAbility to identify uncovered information from hidden data and convert to a story and communicate effectively through visualization\nModelling using R, SAS, Python using Pandas\nWeb scraping using Beautiful soup in Python\nDatabases like Oracle and Microsoft SQL\nExperience in ASP.NET 4.5, C#, 

In [25]:
!pip install flashrank # rerank lib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 97.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


In [26]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from flashrank import Ranker  # Make sure `pip install flashrank` is done

# from langchain_openai import ChatOpenAI

# we have to defne our orignl retrver here to perform rerank
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={"k": 30})

# Step 2: Initialize Flashrank with a reranker model (NOT gte-small)
ranker = Ranker(model_name="ms-marco-MultiBERT-L-12")  # Default model
compressor = FlashrankRerank(model=ranker, top_n=5)   # Rerank to top 5

# compressor = FlashrankRerank(top_n= 5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    """
  "Machine Learning Engineer" OR "Data Scientist" AND (
    ("Python" AND ("Scikit-learn" OR "TensorFlow" OR "PyTorch"))  
    ("SQL" AND ("Spark" OR "Hadoop" OR "ETL"))  
    ("AWS" OR "GCP" OR "Azure" OR "MLOps")  
    ("Tableau" OR "Power BI" OR "data visualization")  
    ("statistical analysis" OR "A/B testing")  
    ("NLP" OR "LLM" OR "GenAI" OR "recommendation systems")  
  )  
  NOT ("intern" OR "student")  
  Years: "3+ years"  
  Location: "Remote" OR "Mumbai" OR "Bangalore" OR "Pune"  
""")
print([doc.metadata["id"] for doc in compressed_docs])

# here eroor because reranl model in inbuilt you dont need to speccify

ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:00<00:00, 272MiB/s]


TypeError: unsupported operand type(s) for /: 'PosixPath' and 'Ranker'

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank

# Initialize your base retriever (using gte-small embeddings)
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={"k": 30})

# Initialize FlashrankRerank - let it handle the Ranker internally
compressor = FlashrankRerank(top_n=5)  # This will automatically use ms-marco-MultiBERT-L-12

# Create compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Run your query
query = "Machine Learning Engineer with Python and 3+ years experience in Bangalore"
compressed_docs = compression_retriever.invoke(query)

# Print results
print([doc.metadata.get("id") for doc in compressed_docs])
#  here errpor due tomssing code below
# import langchain_community.document_compressors.flashrank_rerank as flashrank_rerank
# flashrank_rerank.RerankRequest = RerankRequest
# which s not mentned in lancan exmple

In [33]:
# all corrected rearank code
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from flashrank import Ranker, RerankRequest  # Explicitly import RerankRequest

# Monkey patch the missing reference
import langchain_community.document_compressors.flashrank_rerank as flashrank_rerank
flashrank_rerank.RerankRequest = RerankRequest

# Initialize retriever
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={"k": 30})

# Initialize compressor
compressor = FlashrankRerank(top_n=5)

# Create compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Run query
query =  """
  "Machine Learning Engineer" OR "Data Scientist" AND (
    ("Python" AND ("Scikit-learn" OR "TensorFlow" OR "PyTorch"))  
    ("SQL" AND ("Spark" OR "Hadoop" OR "ETL"))  
    ("AWS" OR "GCP" OR "Azure" OR "MLOps")  
    ("Tableau" OR "Power BI" OR "data visualization")  
    ("statistical analysis" OR "A/B testing")  
    ("NLP" OR "LLM" OR "GenAI" OR "recommendation systems")  
  )  
  NOT ("intern" OR "student")  
  Years: "3+ years"  
  Location: "Remote" OR "Mumbai" OR "Bangalore" OR "Pune"  
"""
compressed_docs = compression_retriever.invoke(query)

# Print results
print([doc.metadata.get("id") for doc in compressed_docs])

2025-05-20 13:59:37.828659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747749577.850671     229 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747749577.857466     229 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

[2, 15, 8, 6, 5]


In [34]:
pprint.pp(compressed_docs)

[Document(metadata={'id': 2, 'relevance_score': 0.9871056, 'Category': 'ENGINEERING', 'start_index': 0}, page_content='ENGINEERING INTERN\nSkills\nC++, Python, MATLAB, Git, Bash, R, SQL (basic). Experienced in Linux/Unix and using high performance computing clusters.\nMachine Learning Tools and Libraries: Scikit-learn, Pandas, Seaborn, matplotlib, TensorFlow (basic). (I built a XGBoost\nmodel that has 77.5% accuracy in the Kaggle Titanic challenge.)\nComputational Fluid Dynamics and Discrete Element Method Codes\nCFD-DEM, OpenFOAM, CFD-ACE+Â®, FluentÂ®, COMSOLÂ®, LAMMPS, and LIGGGHTS.\nReservoir and Fracture Modeling Tools\nCMGÂ® for reservoir simulation; FracProÂ® for fracture simulation and analysis; Saphir for pressure transient analysis.\nExperimental and Statistical Methods\nSEM, AFM, Confocal Microscopy, Regression analysis, Statistical process control, Design of experiments.\nExperience\nENGINEERING INTERN\n \n08/2016\n \nï¼\u200b \n12/2016\n \nCompany Name\n \nState\nProject: D